In [6]:
!pip install -q zhconv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [7]:
import os
if not os.path.exists('dict.txt.big'):
  !wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big

--2024-08-27 11:33:44--  https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big [following]
--2024-08-27 11:33:45--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [application/octet-stream]
Saving to: ‘dict.txt.big’

dict.txt.big        100%[===================>]   8.18M  --.-KB/s    in 0.1s    

2024-08-27 11:33:45 (61.9 MB/s) - ‘dict.txt.big’ saved [8583143/8583143]



In [8]:
#disk usage
#-s顯示目錄總大小，而不是個文件大小
#-h以易讀方式顯示大小
!du -sh 'dict.txt.big'

8.2M	dict.txt.big


In [9]:
import jieba
jieba.set_dictionary('dict.txt.big')

In [10]:
# WikiCorpusDownload https://zh.wikipedia.org/wiki/Wikipedia:%E6%95%B0%E6%8D%AE%E5%BA%93%E4%B8%8B%E8%BD%BD
!wget "https://dumps.wikimedia.org/zhwiki/20240820/zhwiki-20240820-pages-articles-multistream.xml.bz2"

--2024-08-27 11:33:47--  https://dumps.wikimedia.org/zhwiki/20240820/zhwiki-20240820-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3123975901 (2.9G) [application/octet-stream]
Saving to: ‘zhwiki-20240820-pages-articles-multistream.xml.bz2’

s-articles-multistr   0%[                    ]  28.82M  4.67MB/s    eta 10m 35s^C


In [ ]:
!du -sh 'zhwiki-20240820-pages-articles-multistream.xml.bz2'
!echo ''
!md5sum 'zhwiki-20240820-pages-articles-multistream.xml.bz2'
!echo ''
!file 'zhwiki-20240820-pages-articles-multistream.xml.bz2'

3.0G	zhwiki-20240820-pages-articles-multistream.xml.bz2

e3a25857e02c7905f773027c74e1a00f  zhwiki-20240820-pages-articles-multistream.xml.bz2

zhwiki-20240820-pages-articles-multistream.xml.bz2: bzip2 compressed data, block size = 900k


In [ ]:
print(list(jieba.cut("中英夾雜的example，Word2Vec應該很interesting吧?")))

['中', '英', '夾雜', '的', 'example', '，', 'Word2Vec', '應該', '很', 'interesting', '吧', '?']


In [ ]:
import spacy

# 下載語言模組
spacy.cli.download("zh_core_web_sm")  # 下載 spacy 中文模組
spacy.cli.download("en_core_web_sm")  # 下載 spacy 英文模組

nlp_zh = spacy.load("zh_core_web_sm") # 載入 spacy 中文模組
nlp_en = spacy.load("en_core_web_sm") # 載入 spacy 英文模組

STOPWORDS =  nlp_zh.Defaults.stop_words | nlp_en.Defaults.stop_words | set(["\n", "\r\n", "\t", " ", ""])

✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import zhconv #簡to繁
for word in STOPWORDS.copy():
    STOPWORDS.add(zhconv.convert(word, "zh-tw"))
print(len(STOPWORDS))

3006


In [ ]:
# python3.9ver以上才被接受list[str]，ref: https://myapollo.com.tw/blog/python-typing-module/
from typing import List
def preprocess_and_tokenize(text: str, token_min_len: int=1, token_max_len: int=15, lower: bool=True) -> List[str]:
    if lower:
        text=text.lower()

    text=zhconv.convert(text, "zh-tw")
    return [
        token for token in jieba.cut(text, cut_all=False)
        if token_min_len <= len(token) <= token_max_len and token not in STOPWORDS
    ]

In [ ]:
print(preprocess_and_tokenize("中英夾雜的example，Word2Vec應該很interesting吧?北京清华大学"))

['中', '英', '夾雜', 'example', 'word2vec', 'interesting', '北京', '清華大學']


In [ ]:
from gensim.corpora import WikiCorpus
wiki_corpus = WikiCorpus('zhwiki-20240820-pages-articles-multistream.xml.bz2', tokenizer_func=preprocess_and_tokenize, token_min_len=1)

Process InputQueue-2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/gensim/utils.py", line 1302, in run
    self.q.put(wrapped_chunk.pop(), block=True)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 89, in put
    if not self._sem.acquire(block, timeout):
KeyboardInterrupt


In [ ]:
g = wiki_corpus.get_texts()
print(next(g)[:10])
print(next(g)[:10])
print(next(g)[:10])

In [ ]:
generator = wiki_corpus.get_texts()

with open("wiki_seg.txt", "w", encoding='utf-8') as output:
    for texts_num, tokens in enumerate(generator):
        output.write(" ".join(tokens) + "\n")

        if (texts_num + 1) % 100000 == 0:
            print(f"[{str(dt.now()):.19}] 已寫入 {texts_num} 篇斷詞文章")

In [1]:
!wget http://nlp.tmu.edu.tw/word2vec/model/tmunlp_1.6B_WB_100dim_2020v1.bin.gz

--2024-08-27 11:28:42--  http://nlp.tmu.edu.tw/word2vec/model/tmunlp_1.6B_WB_100dim_2020v1.bin.gz
Resolving nlp.tmu.edu.tw (nlp.tmu.edu.tw)... 120.97.67.21
Connecting to nlp.tmu.edu.tw (nlp.tmu.edu.tw)|120.97.67.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 656815448 (626M) [application/octet-stream]
Saving to: ‘tmunlp_1.6B_WB_100dim_2020v1.bin.gz’

tmunlp_1.6B_WB_100d 100%[===================>] 626.39M  4.33MB/s    in 79s     

2024-08-27 11:30:01 (7.95 MB/s) - ‘tmunlp_1.6B_WB_100dim_2020v1.bin.gz’ saved [656815448/656815448]



In [5]:
import gensim
from gensim.models.word2vec import Word2Vec
model=gensim.models.KeyedVectors.load_word2vec_format('tmunlp_1.6B_WB_100dim_2020v1.bin.gz', \
                           unicode_errors='ignore',binary=True)

In [ ]:
%time

from gensim.models import word2vec
import multiprocessing
max_cpu_counts = multiprocessing.cpu_count()
print(f"Use {max_cpu_counts} workers to train Word2Vec (dim={100})")


# 讀取訓練語句
sentences = word2vec.LineSentence(model)

# 訓練模型
model = word2vec.Word2Vec(sentences, size=100, workers=max_cpu_counts)

# 儲存模型
output_model = f"word2vec.zh.{100}.model"
model.save(output_model)